# Neural Network

## Data preperation

In [1]:
import sys
import numpy as np
from numpy import loadtxt
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.compose import make_column_transformer
from sklearn.pipeline import Pipeline

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense
from tensorflow.keras.optimizers import SGD

In [2]:
# import data
data = pd.read_csv (r'bank-additional.csv', sep = ';', engine= 'python')
data = data.head(1000)
length = data.shape[0]
data.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,30,blue-collar,married,basic.9y,no,yes,no,cellular,may,fri,...,2,999,0,nonexistent,-1.8,92.893,-46.2,1.313,5099.1,no
1,39,services,single,high.school,no,no,no,telephone,may,fri,...,4,999,0,nonexistent,1.1,93.994,-36.4,4.855,5191.0,no
2,25,services,married,high.school,no,yes,no,telephone,jun,wed,...,1,999,0,nonexistent,1.4,94.465,-41.8,4.962,5228.1,no
3,38,services,married,basic.9y,no,unknown,unknown,telephone,jun,fri,...,3,999,0,nonexistent,1.4,94.465,-41.8,4.959,5228.1,no
4,47,admin.,married,university.degree,no,yes,no,cellular,nov,mon,...,1,999,0,nonexistent,-0.1,93.200,-42.0,4.191,5195.8,no


In [3]:
# select variables
cats_to_use = ['age', 'default', 'contact', 'month', 'previous', 'poutcome', 'emp.var.rate', 'euribor3m', 'nr.employed', 'y']
data = data[cats_to_use]

# 'age', 'job', 'marital', 'education', 'default', 'housing', 'loan',
#       'contact', 'month', 'day_of_week', 'duration', 'campaign', 'pdays',
#       'previous', 'poutcome', 'emp.var.rate', 'cons.price.idx',
#       'cons.conf.idx', 'euribor3m', 'nr.employed', 'y'

In [4]:
# save lists of categorical and numerical variables
cat_cols = ['default', 'contact', 'month', 'poutcome', 'y']
num_cols = ['age', 'previous', 'emp.var.rate', 'euribor3m', 'nr.employed']

# create column transformer to 1 one-hot-encode cat vars and 2 noralise num vars
ct = make_column_transformer(
    (OneHotEncoder(drop='first'), cat_cols), # drop first column (reference)
    (StandardScaler(), num_cols),
)

# transform base table (pandas df -> numpy array)
base = ct.fit_transform(data)

# convert base table to p.df for ease of use (numpy array -> pandas df)
base_temp = pd.DataFrame(base, columns=ct.get_feature_names_out().tolist())
base_temp

,onehotencoder__default_unknown,onehotencoder__contact_telephone,onehotencoder__month_aug,onehotencoder__month_dec,onehotencoder__month_jul,onehotencoder__month_jun,onehotencoder__month_mar,onehotencoder__month_may,onehotencoder__month_nov,onehotencoder__month_oct,onehotencoder__month_sep,onehotencoder__poutcome_nonexistent,onehotencoder__poutcome_success,onehotencoder__y_yes,standardscaler__age,standardscaler__previous,standardscaler__emp.var.rate,standardscaler__euribor3m,standardscaler__nr.employed
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,-0.933203,-0.362461,-1.286073,-1.341691,-0.881698
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,-0.075480,-0.362461,0.644268,0.709627,0.340863
2,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,-1.409717,-0.362461,0.843959,0.771596,0.834410
3,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,-0.170782,-0.362461,0.843959,0.769858,0.834410
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.686941,-0.362461,-0.154494,0.325078,0.404718
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,-0.266085,1.388557,-0.154494,0.258476,0.404718
996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,-0.933203,-0.362461,-0.154494,0.283959,0.404718
997,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,-0.647295,-0.362461,0.644268,0.709627,0.340863
998,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,-0.361388,-0.362461,0.644268,0.710786,0.340863


## The model

In [5]:
# check list of column names in base table
base_temp.columns.tolist()

['onehotencoder__default_unknown',
 'onehotencoder__contact_telephone',
 'onehotencoder__month_aug',
 'onehotencoder__month_dec',
 'onehotencoder__month_jul',
 'onehotencoder__month_jun',
 'onehotencoder__month_mar',
 'onehotencoder__month_may',
 'onehotencoder__month_nov',
 'onehotencoder__month_oct',
 'onehotencoder__month_sep',
 'onehotencoder__poutcome_nonexistent',
 'onehotencoder__poutcome_success',
 'onehotencoder__y_yes',
 'standardscaler__age',
 'standardscaler__previous',
 'standardscaler__emp.var.rate',
 'standardscaler__euribor3m',
 'standardscaler__nr.employed']

In [6]:
# seperate base table into X and y and convert to numpy array (base pandas df -> y numpy array + X numpy array)
y = base_temp['onehotencoder__y_yes'].values
X = base_temp.drop(columns=['onehotencoder__y_yes']).values

# save and check dimensions of X 
(X_length, X_vars) = X.shape
X_length, X_vars

(1000, 18)

In [7]:
# define model (input layer (X_vars-d) > hidden layer (12-d) > output layer (1-d))
model = Sequential()
model.add(Dense(12, input_dim=X_vars, activation='relu')) # input + hidden layer: 12 nodes + relu (TUNE #NODES!)
model.add(Dense(1, activation='sigmoid')) # output layer: 1 node + sigmoid

# compile model (Adam performs well (source?), AUC for comparison)
model.compile(loss='binary_crossentropy', optimizer='Adam', metrics=[tf.keras.metrics.AUC()]) # ['accuracy']

# fit model on the dataset
model.fit(X, y, epochs=100, batch_size=10) # TUNE #EPOCHS, BATCH_SIZE!

# evaluate model, print AUC
_, accuracy = model.evaluate(X, y, verbose=0) # what is verbose?
print('Accuracy: %.2f' %(accuracy*100))

Epoch 1/100
100/100 [==============================] - 0s 2ms/step - loss: 0.5985 - auc: 0.6757
Epoch 2/100
100/100 [==============================] - 0s 2ms/step - loss: 0.4260 - auc: 0.6712
Epoch 3/100
100/100 [==============================] - 0s 2ms/step - loss: 0.3774 - auc: 0.6639
Epoch 4/100
100/100 [==============================] - 0s 4ms/step - loss: 0.3501 - auc: 0.6749
Epoch 5/100
100/100 [==============================] - 0s 4ms/step - loss: 0.3265 - auc: 0.6897
Epoch 6/100
100/100 [==============================] - 0s 4ms/step - loss: 0.3075 - auc: 0.7113
Epoch 7/100
100/100 [==============================] - 0s 4ms/step - loss: 0.2946 - auc: 0.7399
Epoch 8/100
100/100 [==============================] - 0s 3ms/step - loss: 0.2857 - auc: 0.7518
Epoch 9/100
100/100 [==============================] - 0s 4ms/step - loss: 0.2782 - auc: 0.7607
Epoch 10/100
100/100 [==============================] - 0s 4ms/step - loss: 0.2717 - auc: 0.7719
Epoch 11/100
100/100 [=================

100/100 [==============================] - 0s 4ms/step - loss: 0.2267 - auc: 0.8371
Epoch 86/100
100/100 [==============================] - 0s 4ms/step - loss: 0.2263 - auc: 0.8377
Epoch 87/100
100/100 [==============================] - 0s 4ms/step - loss: 0.2259 - auc: 0.8382
Epoch 88/100
100/100 [==============================] - 0s 3ms/step - loss: 0.2258 - auc: 0.8380
Epoch 89/100
100/100 [==============================] - 0s 2ms/step - loss: 0.2260 - auc: 0.8399
Epoch 90/100
100/100 [==============================] - 0s 3ms/step - loss: 0.2252 - auc: 0.8381
Epoch 91/100
100/100 [==============================] - 0s 3ms/step - loss: 0.2260 - auc: 0.8400
Epoch 92/100
100/100 [==============================] - 0s 3ms/step - loss: 0.2252 - auc: 0.8400
Epoch 93/100
100/100 [==============================] - 0s 2ms/step - loss: 0.2245 - auc: 0.8394
Epoch 94/100
100/100 [==============================] - 0s 3ms/step - loss: 0.2248 - auc: 0.8406
Epoch 95/100
100/100 [=====================

In [8]:
# OLD NN CODE
#Changing pandas dataframe to numpy array
X = features.values
y = y.values

#Normalizing the data
sc = StandardScaler()
X = sc.fit_transform(X)

train_features, test_features, train_targets, test_targets = train_test_split(X, y, test_size=0.2,
                                                                                  random_state=23)

# Neural network
model = Sequential()
model.add(Dense(16, input_dim=54, activation='relu'))
model.add(Dense(12, activation='relu'))
model.add(Dense(1, activation='softmax'))

opt = SGD(lr=0.01)
model.compile(loss = "binary_crossentropy", optimizer = opt, metrics=['accuracy'])

history = model.fit(train_features, train_targets, epochs=100, batch_size=64)

NameError: name 'features' is not defined

In [ ]:
# CODE FROM BOOSTED TREES
predictions = model.predict(test_features)


# Use score method to get accuracy of model
accuracy = metrics.accuracy_score(test_targets, predictions)
print("Accuracy: ", + np.round(accuracy , 3))

print("Confusion Matrix:")
print(confusion_matrix(test_targets, predictions))

print("Classification Report")
print(classification_report(test_targets, predictions))
    
#Beginning the plotting of ROC-curve
pred_prob = classifier.predict_proba(test_features)
fpr, tpr, thresh = roc_curve(test_targets, pred_prob[:,1], pos_label=1)
    
#Plot roc curves
plt.plot(fpr, tpr, linestyle='--',color='orange', label='SVM')

# title
plt.title('ROC curve')
# x label
plt.xlabel('False Positive Rate')
# y label
plt.ylabel('True Positive rate')

plt.legend(loc='best')
plt.savefig('ROC',dpi=300)
plt.show();
    
#AUC Score
auc_score = roc_auc_score(test_targets, pred_prob[:,1])
print("AUC Score: " + str(np.round(auc_score , 3)))
    
#Log-loss function
print("Log-Loss: " + str(np.round(log_loss(test_targets, predictions),3)))